In [1]:
"""
참고교재: Deep Learning for Time Series Forecasting - Brownlee
https://machinelearningmastery.com/how-to-develop-multilayer-perceptron-models-for-time-series-forecasting/
2022.07
"""

#==============================================#
#====== MLPs for Time Series Forecasting ======#
#==============================================#

"""
1. Univariate MLP Models
2. Multivariate MLP Models
3. Multi-Step MLP Models
4. Multivariate Multi-Step MLP Models
"""

'\n1. Univariate MLP Models\n2. Multivariate MLP Models\n3. Multi-Step MLP Models\n4. Multivariate Multi-Step MLP Models\n'

In [2]:
##---- 1. Univariate MLP Models ----#

# 필요한 라이브러리 불러오기
from numpy import array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 

In [3]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

# choose a number of time steps
n_steps = 3

# split into samples
X, y = split_sequence(raw_seq, n_steps)

In [4]:
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])
    
"""
[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70

[50 60 70] 80
[60 70 80] 90
"""

[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90


'\n[10 20 30] 40\n[20 30 40] 50\n[30 40 50] 60\n[40 50 60] 70\n\n[50 60 70] 80\n[60 70 80] 90\n'

In [5]:
# define model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_steps))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [6]:
# fit model
model.fit(X, y, epochs=2000, verbose=0)

In [7]:
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps))  # x_input.shape: (1,3)
yhat = model.predict(x_input, verbose=0)
print(yhat)
# [[101.22577]]

[[101.908615]]


In [8]:
##---- 2. Multivariate MLP Models ----#
"""
1. Multiple Input Series
2. Multiple Parallel Series
"""

##---- 2.1 Multiple Input Series ----#

# 필요한 라이브러리 불러오기
from numpy import array
from numpy import hstack
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import concatenate

In [9]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [10]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))

# choose a number of time steps
n_steps = 3

# convert into input/output
X, y = split_sequences(dataset, n_steps)
print(X.shape, y.shape)
# (7, 3, 2) (7,)

(7, 3, 2) (7,)


In [11]:
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])
"""
[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185
"""    

[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


'\n[[10 15]\n [20 25]\n [30 35]] 65\n[[20 25]\n [30 35]\n [40 45]] 85\n[[30 35]\n [40 45]\n [50 55]] 105\n[[40 45]\n [50 55]\n [60 65]] 125\n[[50 55]\n [60 65]\n [70 75]] 145\n[[60 65]\n [70 75]\n [80 85]] 165\n[[70 75]\n [80 85]\n [90 95]] 185\n'

In [12]:
# separate input data
X1 = X[:, :, 0]
X2 = X[:, :, 1]

# summarize the data
for i in range(len(X)):
	print(X1[i], X2[i])
"""
[10 20 30] [15 25 35]
[20 30 40] [25 35 45]
[30 40 50] [35 45 55]
[40 50 60] [45 55 65]
[50 60 70] [55 65 75]
[60 70 80] [65 75 85]
[70 80 90] [75 85 95]
"""
    

[10 20 30] [15 25 35]
[20 30 40] [25 35 45]
[30 40 50] [35 45 55]
[40 50 60] [45 55 65]
[50 60 70] [55 65 75]
[60 70 80] [65 75 85]
[70 80 90] [75 85 95]


'\n[10 20 30] [15 25 35]\n[20 30 40] [25 35 45]\n[30 40 50] [35 45 55]\n[40 50 60] [45 55 65]\n[50 60 70] [55 65 75]\n[60 70 80] [65 75 85]\n[70 80 90] [75 85 95]\n'

In [13]:
# first input model
visible1 = Input(shape=(n_steps,))
dense1 = Dense(100, activation='relu')(visible1)

# second input model
visible2 = Input(shape=(n_steps,))
dense2 = Dense(100, activation='relu')(visible2)

# merge input models
merge = concatenate([dense1, dense2])
output = Dense(1)(merge)
model = Model(inputs=[visible1, visible2], outputs=output)
model.compile(optimizer='adam', loss='mse')

In [14]:
# fit model
model.fit([X1, X2], y, epochs=2000, verbose=0)

# demonstrate prediction
x_input = array([[80, 85], [90, 95], [100, 105]])
x1 = x_input[:, 0].reshape((1, n_steps))
x2 = x_input[:, 1].reshape((1, n_steps))
yhat = model.predict([x1, x2], verbose=0)
print(yhat)
# [[206.28448]]

[[205.51033]]


In [15]:
##---- 2.2 Multiple Parallel Series ----#

# 필요한 라이브러리 불러오기
from numpy import array
from numpy import hstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [16]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [17]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))

# choose a number of time steps
n_steps = 3

# convert into input/output
X, y = split_sequences(dataset, n_steps)
print(X.shape, y.shape)
# (6, 3, 3) (6, 3)

(6, 3, 3) (6, 3)


In [18]:
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])
"""
[[10 15 25]
 [20 25 45]
 [30 35 65]] [40 45 85]
[[20 25 45]
 [30 35 65]
 [40 45 85]] [ 50  55 105]
[[ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]] [ 60  65 125]
[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]] [ 70  75 145]
[[ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]] [ 80  85 165]
[[ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]] [ 90  95 185]
"""
    

[[10 15 25]
 [20 25 45]
 [30 35 65]] [40 45 85]
[[20 25 45]
 [30 35 65]
 [40 45 85]] [ 50  55 105]
[[ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]] [ 60  65 125]
[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]] [ 70  75 145]
[[ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]] [ 80  85 165]
[[ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]] [ 90  95 185]


'\n[[10 15 25]\n [20 25 45]\n [30 35 65]] [40 45 85]\n[[20 25 45]\n [30 35 65]\n [40 45 85]] [ 50  55 105]\n[[ 30  35  65]\n [ 40  45  85]\n [ 50  55 105]] [ 60  65 125]\n[[ 40  45  85]\n [ 50  55 105]\n [ 60  65 125]] [ 70  75 145]\n[[ 50  55 105]\n [ 60  65 125]\n [ 70  75 145]] [ 80  85 165]\n[[ 60  65 125]\n [ 70  75 145]\n [ 80  85 165]] [ 90  95 185]\n'

In [19]:
# flatten input
n_input = X.shape[1] * X.shape[2]
X = X.reshape((X.shape[0], n_input))
n_output = y.shape[1]

# define model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_input))
model.add(Dense(n_output))
model.compile(optimizer='adam', loss='mse')

In [20]:
# fit model
model.fit(X, y, epochs=2000, verbose=0)

# demonstrate prediction
x_input = array([[70,75,145], [80,85,165], [90,95,185]])
x_input = x_input.reshape((1, n_input))
yhat = model.predict(x_input, verbose=0)
print(yhat)
# [[101.07046  107.014305 208.90628 ]]

[[100.232124 105.386765 206.14807 ]]


In [21]:
##---- 3. Multi-Step MLP Models ----#
"""
univariate multi-step vector-output mlp example
"""

# 필요한 라이브러리 불러오기
from numpy import array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [22]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [23]:
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

# choose a number of time steps
n_steps_in, n_steps_out = 3, 2

# split into samples
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)

In [24]:
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])
"""
[10 20 30] [40 50]
[20 30 40] [50 60]
[30 40 50] [60 70]
[40 50 60] [70 80]
[50 60 70] [80 90]
"""

[10 20 30] [40 50]
[20 30 40] [50 60]
[30 40 50] [60 70]
[40 50 60] [70 80]
[50 60 70] [80 90]


'\n[10 20 30] [40 50]\n[20 30 40] [50 60]\n[30 40 50] [60 70]\n[40 50 60] [70 80]\n[50 60 70] [80 90]\n'

In [25]:
# define model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_steps_in))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')

In [26]:
# fit model
model.fit(X, y, epochs=2000, verbose=0)

In [27]:
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps_in))
yhat = model.predict(x_input, verbose=0)
print(yhat)
# [[105.00911 116.91802]]  

[[105.04342 116.83817]]


In [28]:
##---- 4. Multivariate Multi-Step MLP Models ----#
"""
1. Multiple Input Multi-Step Output
2. Multiple Parallel Input and Multi-Step Output
"""

##---- 4.1 Multiple Input Multi-Step Output ----#

# 필요한 라이브러리 불러오기
from numpy import array
from numpy import hstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [29]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [30]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))

# choose a number of time steps
n_steps_in, n_steps_out = 3, 2

In [31]:
# convert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
print(X.shape, y.shape)
for i in range(len(X)):
	print(X[i], y[i])
"""
(6, 3, 2) (6, 2)
[[10 15]
 [20 25]
 [30 35]] [65 85]
[[20 25]
 [30 35]
 [40 45]] [ 85 105]
[[30 35]
 [40 45]
 [50 55]] [105 125]
[[40 45]
 [50 55]
 [60 65]] [125 145]
[[50 55]
 [60 65]
 [70 75]] [145 165]
[[60 65]
 [70 75]
 [80 85]] [165 185]
"""

(6, 3, 2) (6, 2)
[[10 15]
 [20 25]
 [30 35]] [65 85]
[[20 25]
 [30 35]
 [40 45]] [ 85 105]
[[30 35]
 [40 45]
 [50 55]] [105 125]
[[40 45]
 [50 55]
 [60 65]] [125 145]
[[50 55]
 [60 65]
 [70 75]] [145 165]
[[60 65]
 [70 75]
 [80 85]] [165 185]


'\n(6, 3, 2) (6, 2)\n[[10 15]\n [20 25]\n [30 35]] [65 85]\n[[20 25]\n [30 35]\n [40 45]] [ 85 105]\n[[30 35]\n [40 45]\n [50 55]] [105 125]\n[[40 45]\n [50 55]\n [60 65]] [125 145]\n[[50 55]\n [60 65]\n [70 75]] [145 165]\n[[60 65]\n [70 75]\n [80 85]] [165 185]\n'

In [32]:
# flatten input
n_input = X.shape[1] * X.shape[2]
X = X.reshape((X.shape[0], n_input))

# define model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_input))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')

In [33]:
# fit model
model.fit(X, y, epochs=2000, verbose=0)

In [34]:
# demonstrate prediction
x_input = array([[70, 75], [80, 85], [90, 95]])
x_input = x_input.reshape((1, n_input))
yhat = model.predict(x_input, verbose=0)
print(yhat)
# [[186.61156 207.68965]]

[[184.90797 206.3912 ]]


In [35]:
##---- 4.2 Multiple Parallel Input and Multi-Step Output ----#

# 필요한 라이브러리 불러오기
from numpy import array
from numpy import hstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [36]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [37]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))

# choose a number of time steps
n_steps_in, n_steps_out = 3, 2

In [38]:
# convert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])
"""
(5, 3, 3) (5, 2, 3)
[[10 15 25]
 [20 25 45]
 [30 35 65]] [[ 40  45  85]
 [ 50  55 105]]
[[20 25 45]
 [30 35 65]
 [40 45 85]] [[ 50  55 105]
 [ 60  65 125]]
[[ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]] [[ 60  65 125]
 [ 70  75 145]]
[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]] [[ 70  75 145]
 [ 80  85 165]]
[[ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]] [[ 80  85 165]
 [ 90  95 185]]
"""

(5, 3, 3) (5, 2, 3)
[[10 15 25]
 [20 25 45]
 [30 35 65]] [[ 40  45  85]
 [ 50  55 105]]
[[20 25 45]
 [30 35 65]
 [40 45 85]] [[ 50  55 105]
 [ 60  65 125]]
[[ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]] [[ 60  65 125]
 [ 70  75 145]]
[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]] [[ 70  75 145]
 [ 80  85 165]]
[[ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]] [[ 80  85 165]
 [ 90  95 185]]


'\n(5, 3, 3) (5, 2, 3)\n[[10 15 25]\n [20 25 45]\n [30 35 65]] [[ 40  45  85]\n [ 50  55 105]]\n[[20 25 45]\n [30 35 65]\n [40 45 85]] [[ 50  55 105]\n [ 60  65 125]]\n[[ 30  35  65]\n [ 40  45  85]\n [ 50  55 105]] [[ 60  65 125]\n [ 70  75 145]]\n[[ 40  45  85]\n [ 50  55 105]\n [ 60  65 125]] [[ 70  75 145]\n [ 80  85 165]]\n[[ 50  55 105]\n [ 60  65 125]\n [ 70  75 145]] [[ 80  85 165]\n [ 90  95 185]]\n'

In [39]:
# flatten input
n_input = X.shape[1] * X.shape[2]
X = X.reshape((X.shape[0], n_input))

# flatten output
n_output = y.shape[1] * y.shape[2]
y = y.reshape((y.shape[0], n_output))

In [40]:
# define model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_input))
model.add(Dense(n_output))
model.compile(optimizer='adam', loss='mse')

In [41]:
# fit model
model.fit(X, y, epochs=2000, verbose=0)

In [42]:
# demonstrate prediction
x_input = array([[60, 65, 125], [70, 75, 145], [80, 85, 165]])
x_input = x_input.reshape((1, n_input))
yhat = model.predict(x_input, verbose=0)
print(yhat)
# [[ 89.76276   95.424736 185.5788   100.632195 106.121895 205.4496  ]]

[[ 91.17711   97.55464  186.17603  100.80014  105.677864 209.32098 ]]
